In [1]:
# from selenium.webdriver import Chrome
from bs4 import BeautifulSoup
import pymongo
# import datetime
# import time
import pandas as pd
import numpy as np
# import re
import warnings
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import string
%matplotlib inline
import matplotlib.pyplot as plt

%reload_ext autoreload
%autoreload 2

In [2]:
mc = pymongo.MongoClient()
db = mc['chordify']
raw_html = db['raw_html']
parsed_songs_db = db["parsed_songs"]

In [ ]:
html_docs = list(raw_html.find())

In [ ]:
raw_html.find().count()

In [ ]:
parsed_songs = list(parsed_songs_db.find())

In [ ]:
a = parsed_songs[4444]['_id']

In [ ]:
str(a)

In [ ]:
parsed_songs[4444]

In [ ]:
parsed_songs_db.find().count()

In [ ]:
def get_phrase_for_chord(one_parsed_song):

    song_id = str(one_parsed_song['_id'])
    word_list = one_parsed_song['words']

    one_song_chord_phrase_tuples = []

    for chord_set in one_parsed_song['chord_idxs']:
        word_idx = chord_set[0]
        chord_name = chord_set[1]
        phrase = ' '.join(word_list[(word_idx - 8):(word_idx + 3)])
        one_song_chord_phrase_tuples.append((chord_name, phrase, song_id))

    return one_song_chord_phrase_tuples


def get_all_phrase_chord_tuples(parsed_songs):

    chord_phrase_tuples = []

    for song in parsed_songs:
        one_song_chord_phrase_tuples = get_phrase_for_chord(song)
        chord_phrase_tuples.extend(one_song_chord_phrase_tuples)

    return chord_phrase_tuples

In [ ]:
pct = get_all_phrase_chord_tuples(parsed_songs[456:459])

In [ ]:
pct

In [ ]:
def make_phrase_is_minor_list(chord_phrase_tuples):

    phrases = []
    chords = []
    is_minor = []
    song_ids = []

    for chord_phrase_tup in chord_phrase_tuples:
        phrases.append(chord_phrase_tup[1])
        chords.append(chord_phrase_tup[0])
        is_minor.append('m' in chord_phrase_tup[0])
        song_ids.append(chord_phrase_tup[2])

    return phrases, chords, is_minor, song_ids

good_chords = ['A', 'C', 'D', 'G']

def get_list_of_chords(chord_phrase_tuples):
    is_A = []
    is_C = []
    is_D = []
    is_G = []
    other_chord = []
    for tup in chord_phrase_tuples:
        is_A.append(tup[0] == 'A')
        is_C.append(tup[0] == 'C')
        is_D.append(tup[0] == 'D')
        is_G.append(tup[0] == 'G')
        other_chord.append(tup[0] not in good_chords)
    return is_A, is_C, is_D, is_G, other_chord

def make_phrase_chord_df(phrases, chords, is_minor, is_A, is_C, is_D, is_G, other_chord):
    df = pd.DataFrame({
        'chords': chords,
        'A': is_A,
        'C': is_C,
        'D': is_D,
        'G': is_G,
        'other_chord': other_chord,
        'is_minor': is_minor,
        'words': phrases
    })
    return df


In [ ]:
is_A, is_C, is_D, is_G, other_chord = get_list_of_chords(pct)

In [ ]:
phrases, chords, is_minor, song_ids = make_phrase_is_minor_list(pct)

In [ ]:
song_ids

In [ ]:
make_phrase_chord_df(phrases, chords, is_minor, is_A, is_C, is_D, is_G, other_chord)

In [ ]:
def make_phrase_is_minor_list(chord_phrase_tuples):

    phrases = []
    chords = []
    is_minor = []
    song_ids = []

    for chord_phrase_tup in chord_phrase_tuples:
        phrases.append(chord_phrase_tup[1])
        chords.append(chord_phrase_tup[0])
        is_minor.append('m' in chord_phrase_tup[0])
        song_ids.append(chord_phrase_tup[2])

    return phrases, chords, is_minor, song_ids


good_chords = ['A','B','C','D','E','F','G','G#','A7','D7','Em','Am','Bm','Dm','Bb']

def get_list_of_chords(chord_phrase_tuples):
    is_A = []
    is_B = []
    is_C = []
    is_D = []
    is_E = []
    is_F = []
    is_G = []
    is_A7 = []
    is_D7 = []
    is_Em = []
    is_Am = []
    is_Bm = []
    is_Dm = []
    is_Bb = []
    other_chord = []
    for tup in chord_phrase_tuples:
        is_A.append(tup[0] == 'A')
        is_B.append(tup[0] == 'B')
        is_C.append(tup[0] == 'C')
        is_D.append(tup[0] == 'D')
        is_E.append(tup[0] == 'E')
        is_F.append(tup[0] == 'F')
        is_G.append(tup[0] == 'G' or tup[0] == 'G#')
        is_A7.append(tup[0] == 'A7')
        is_D7.append(tup[0] == 'D7')
        is_Em.append(tup[0] == 'Em')
        is_Am.append(tup[0] == 'Am')
        is_Bm.append(tup[0] == 'Bm')
        is_Dm.append(tup[0] == 'Dm')
        is_Bb.append(tup[0] == 'Bb')
        other_chord.append(tup[0] not in good_chords)
    return is_A, is_B, is_C, is_D, is_E, is_F, is_G, is_A7, is_D7, is_Em, is_Am, is_Bm, is_Bb, is_Dm, other_chord


def make_phrase_chord_df(chord_phrase_tuples):

    phrases, chords, is_minor, song_ids = make_phrase_is_minor_list(chord_phrase_tuples)

    is_A, is_B, is_C, is_D, is_E, is_F, is_G, is_A7, is_D7, is_Em, is_Am, is_Bm, is_Bb, is_Dm, other_chord = get_list_of_chords(chord_phrase_tuples)

    df = pd.DataFrame({
        'song_id': song_ids,
        'chord': chords,
        'A': is_A,
        'B': is_B,
        'C': is_C,
        'D': is_D,
        'E': is_E,
        'F': is_F,
        'G': is_G,
        'A7': is_A7,
        'D7': is_D7,
        'Em': is_Em,
        'Am': is_Am,
        'Bm': is_Bm,
        'Dm': is_Dm,
        'Bb': is_Bb,
        'other_chord': other_chord,
        'is_minor': is_minor,
        'words': phrases
    })

    return df

In [ ]:
pct_more = get_all_phrase_chord_tuples(parsed_songs[1111:1357])
pct_more

In [ ]:
df_c = make_phrase_chord_df(pct_more)
df_c

In [ ]:
df_c['chord'].value_counts()

In [ ]:
ids = df_c['song_id']
ids

In [ ]:
type(set(ids))

In [ ]:
tr, te = train_test_split(list(set(ids)))

In [ ]:
len(tr)

In [ ]:
len(te)

In [ ]:
len(set(ids))

In [ ]:
len(df_c.song_id)

In [ ]:
df_train = df_c[df_c.song_id.isin(tr)]

In [ ]:
df_test = df_c[df_c.song_id.isin(te)]

In [ ]:
df_test.shape

In [ ]:
df_c.shape

In [ ]:
df_train.shape

In [ ]:
df_train['other_chord'].value_counts()

In [ ]:
tfidf = TfidfVectorizer()
logistic = LogisticRegression()

In [ ]:
chord_list = ['A','B','C','D','E','F','G','A7','D7','Em','Am','Bm','Dm','Bb']

def find_best_chord(df_train, df_test):
    
    chord_prob_dict = {}

    for chord in chord_list:

         logistic = LogisticRegression()
        
        X_train = df_train['words']
        y_train = df_train[chord]
        
#         X_test = df_test['words']
#         y_test = df_test[chord]
        
        tfidf.fit(X_train)
        tr_matrix = tfidf.transform(X_train)
        
        logistic.fit(tr_matrix, y_train)
        
        te_matrix = tfidf.transform(X_test)
        
        y_hat_log = logistic.predict_proba(te_matrix)
        is_chord = logistic.predict(te_matrix)

        chord_prob_dict[chord] = (y_hat_log[0][1], is_chord)

    print(chord_prob_dict)    
    best_chord = max(chord_prob_dict.items(), key=lambda k: k[1])

    return best_chord 


In [ ]:
find_best_chord(df_train, df_test)

In [ ]:
from src.LogisticChordAnalyzer import LogisticChordAnalyzer

In [ ]:
lca = LogisticChordAnalyzer(tfidf, logistic)

In [ ]:
lca.predict(entry)

In [ ]:
lmcp = LogisticMinorChordPredictor(tfidf, logistic)

In [ ]:
lmcp.predict(entry)

In [ ]:
parsed_songs = list(parsed_songs_db.find())[25:75]

tfidf = TfidfVectorizer()
logistic = LogisticRegression()

phrases_all, chords_all, is_minor_all = get_full_phrase_is_minor_list(parsed_songs)

df_by_line = make_phrase_chord_df(phrases_all, chords_all, is_minor_all)


words_train, words_test, is_minor_train, is_minor_test = train_test_split(df_by_line['words'], df_by_line['is_minor'])

In [ ]:
from src.logistic_by_line import get_full_phrase_is_minor_list, make_phrase_chord_df

In [ ]:
words_train

In [8]:
from src.LogisticChordAnalyzer import LogisticChordAnalyzer

In [ ]:
from src import logstic_many_chords

In [ ]:
tfidf=logistic_many_chords.tfidf

In [13]:
chord_list = ['A','B','C','D','E','F','G','A7','D7','Em','Am','Bm','Dm','Bb']
chord_model_dict = {}
chord_prob_dict = {}


In [14]:
lca = LogisticChordAnalyzer(TfidfVectorizer, LogisticRegression, chord_list, chord_model_dict, chord_prob_dict)

In [ ]:
tfidf

In [15]:
lca.predict('beautiful terrible thunderstorm trucks mama heartbreak love kill find chase')

TypeError: transform() missing 1 required positional argument: 'raw_documents'

In [22]:
pwd

'/Users/emilynaftalin/galvanize/dsi/capstone/Guitar-Chord-Generator'

In [23]:
%run src.LogisticChordAnalyzer.py

ERROR:root:File `'src.LogisticChordAnalyzer.py'` not found.
